In [ ]:
import numba
import numpy
from numba import cuda
x_test = numpy.load('../.keras/datasets/mnist.npz')['x_test']


In [ ]:
@cuda.jit('void(int32[:,:,:], int32[:,:])')
def divergence_explicit(data, divergence):    
    i, j = numba.cuda.grid(2)
    for y in range(28):
        for x in range(28):
            divergence[i, j] += (data[i, y, x] - data[j, y, x]) ** 2

In [ ]:
divergence = numpy.zeros((10000, 10000), dtype=numpy.int32)
%time divergence_explicit[(625,625),(16,16)](x_test.astype(numpy.int32), divergence)

In [ ]:
maxindex = numpy.argmax(divergence)
maxi = maxindex // 10000
maxj = maxindex % 10000

In [ ]:
divergence[maxi, maxj]

In [ ]:
from numba import cuda
@cuda.jit('void(uint8[:,:,:], uint64[:])')
def divergence(data, output):
    sharedmem = numba.cuda.shared.array((1), numba.uint64)
    localmem = numba.cuda.local.array((3), numba.int32)
    finallocal = numba.cuda.local.array((1), numba.uint64)
    localmem[0] = 0
    if cuda.threadIdx.x and cuda.threadIdx.y == 0:    
        sharedmem[0] = 0
        
    cuda.syncthreads()
    i, j = numba.cuda.grid(2)
    
    for y in range(28):
        for x in range(28):
            localmem[1] = data[i, y, x]
            localmem[2] = data[j, y, x]
            localmem[1] -= localmem[2]
            localmem[1] **= 2

            localmem[0] += localmem[1]
                
    finallocal[0] = localmem[0]
    finallocal[0] *= 4294967296
    finallocal[0] += i * 65536
    finallocal[0] += j
    
    cuda.atomic.max(sharedmem, 0, finallocal[0])
    cuda.syncthreads()
    if cuda.threadIdx.x == 0 and cuda.threadIdx.y == 0:
        cuda.atomic.max(output, 0, sharedmem[0])
    
    

In [ ]:
output = numpy.array(0, dtype=numpy.uint64)
%time divergence[(625,625),(16,16)](x_test, output)
output = output.astype(dtype=numpy.int64)

div = numpy.right_shift(output, 32)
maxi = numpy.bitwise_and(numpy.right_shift(output, 16), 65535)
maxj = numpy.bitwise_and(output, 65535)

In [ ]:
import cupy

In [ ]:
x_test2 = cupy.array(x_test).reshape((10000,28*28))
divergence = cupy.zeros((10000,10000),dtype=numpy.int32)
row_kernel = cupy.ReductionKernel('uint8 x, uint8 y', 'int32 z',
                                  '((int) x - (int) y) * ((int) x - (int) y)',
                                  'a + b',
                                  'z = a',
                                  '0',
                                  'row_kernel')



In [ ]:
%time divergence[:, :] = row_kernel(x_test2.reshape((10000, 1, 28*28)), x_test2.reshape((1, 10000, 28*28)), axis=(2))

In [ ]:
%time cupy.asnumpy(divergence)